In [ ]:
from music21 import *

## Intervals

### measuring intervals

The interval module calculates intervals. Notice that since an interval is a distance, it always needs two notes (start, stop) to calculate the distance. We don't know, for example, that the interval is sequential (i.e., melodic) or simultaneous (i.e., harmonic).

In [ ]:
#example of how to measure the interval between two notes:
n1 = note.Note('C4')
n2 = note.Note('F5')
i = (interval.Interval(n1, n2))
i

In [ ]:
# return the simple equivalent of the compound interval (collapse to less than 8ve)
print(i.name+",", i.simpleNiceName)

We can get the direction of the interval (assuming it is melodic, of course, otherwise it wouldn't make much sense).

In [ ]:
i.direction

Recall that enharmonics are not the same when it comes to intervals, so C to F# is a different diatonic interval from C to Gb:

In [ ]:
int1 = (interval.Interval(note.Note('C4'),note.Note('F#4')))
int2 = (interval.Interval(note.Note('C4'),note.Note('G-4')))

In [ ]:
print(int1.name, int2.name)

Of course, if you only have MIDI numbers, then you may get the wrong intervals sometimes. Since we don't know whether midi value 66 is F# or Gb, music21 will take a "good guess". 

In [ ]:
#two different ways of setting a note by midi number. The first is obviously simpler:
midi1 = note.Note(60)
midi2 = note.Note(pitch.Pitch(66))
#calculate the interval:
i = interval.Interval(midi1, midi2)
i

In this case we may want to return the interval size in semitones:

In [ ]:
i.semitones

Intervals can be categorized as consonant if their simple interval is one of any imperfect consonance (3rds, 6ths), or perfect consonance (Unisons, P5ths)

Notice that fourths are not considered 'consonant' here. Music history may be your guide as to whether you should rely on this function or create your own.

In [ ]:
i.isConsonant()

### melodic intervals

Let's read in a file and calculate all the melodic intervals.

In [ ]:
myfile = 'musi4843-data/KernFiles/Essen/europa/england/england1.krn'

In [ ]:
folk = converter.parse(myfile)

In [ ]:
folk.show()

Let's calculate all the melodic intervals and put them in a stream

In [ ]:
#create an empty list to hold intervals
intlist = []
#iterate through each pair of notes in the piece and store consecutive intervals
for i in folk.recurse().getElementsByClass('Note'):
    if i.next('Note') is None:
        continue
    thisint = interval.Interval(i, i.next('Note'))
    intlist.append(thisint)

intlist
    


the `Counter` is a handy tool from the `collections` python module (not a music21 tool.)

In [ ]:
from collections import Counter

In [ ]:
Counter([i.name for i in intlist])

In [ ]:
Counter([i.isConsonant() for i in intlist])

We'll come back to the `pandas` library next week, but you can also pass any list into pandas as a Series object to take advantage of the many useful pandas functions such as `apply` and `value_counts`

In [ ]:
import pandas as pd
intseries = pd.Series(intlist)

In [ ]:
intseries.apply(lambda x: x.diatonic.directedSemiSimpleName).value_counts()

In [ ]:
intseries.apply(lambda x: x.diatonic.directedSemiSimpleName).value_counts().plot.bar()

### Exercise 1 

in the Essen/europa/danmark collection compare the melodic intervals of "danmark4.krn" and "danmark5.krn". Which one has a diminished fifth?

## Iterating through local files 
Ensure that you have imported `music21` and `os` packages. Here is a helper function to load several files into a list for bulk (or sequential) analysis:

In [ ]:
import os

In [ ]:
# variable "mydir" must be a path to the directory with the files you want to parse - make sure to END WITH FORWARD SLASH
def loadFileHelper(mydir):
    all_files = []
    for root, dirs, files in os.walk(mydir):
         for file_name in files:
            if file_name.endswith(('.mid','.krn','.mxl','.xml','mei')):
                print(file_name) #un-comment if you want the function to printout all the filenames
                all_files.append(converter.parse(mydir+file_name))
    return(all_files)


In [ ]:
myfiles = loadFileHelper('musi4843-data/KernFiles/Essen/europa/danmark/')

### Exercise 2:
Now you can answer the question: Do any of these files have melodic intervals greater than a tritone? If so, which ones?

## Chords and Appending Parts

In [ ]:
s = corpus.parse('bwv66.6')
s.show()

In [ ]:
sChords = s.chordify()
sChords.show()

`sChords` is a part which has measures, which have the chords (and key signatures, etc.) inside. To get to the chords, we could "flatten" the score but that will remove the hierarchy. A safer alternative is to use the `recurse()` function

Perhaps we can take each chord, make a reduction where appropriate, and put this in a separate part to join to the original score itself

In [ ]:
#Be caerful, you may want to make a copy before you do in-place changes...
#iterate through each measure object in the score
for m in sChords.getElementsByClass('Measure'):
    #iterate through each object in the measure
    for c in m.getElementsByClass('Chord'):
        #iterate through each chord, convert to closed position and put in 4th octave
        c.closedPosition(forceOctave=4, inPlace=True) #inplace permanently changes the chord

In [ ]:
sChords.show()

In [ ]:
#recall our score object is saved as "s"
#Add our chord reduction to the score obejct
sChords.partName = 'Reduction'
sChords.partAbbreviation = 'Harm'
s.insert(0, sChords) #syntax: location to put item; item to insert
s.show()


### Exercise 3: 
Make a chord arrangement of a piece that is a bit more complex and compare the results. Try Mozart's K458 in the built-in corpus ('mozart/k458/movement1.mxl')

## Rhythm analysis

Let's look at the distribution of beat information in the same set of Essen songs:

Note: one of them didn't import the time signature correctly! You can't get beat info without a time signature.

In [ ]:
beatOnsets = []
for f in myfiles:
    if f.recurse().getElementsByClass('TimeSignature'):
        for n in f.flat.notes:
            beatOnsets.append(n.beat)
        

In [ ]:
from collections import Counter
Counter(beatOnsets)

Note the "fractions" above are incorrect (in that those items never appear in the score). Not sure why this is happening. It's probably poor parsing of kern files. (if you plan to use kern files probably better to use Humdrum in many cases anyway for analysis.) The equivalent Humdrum function is:

`beat *.krn | ridx -GLIdM | sort | uniq -c`

In general, there are often many issues with music21's offsets (i.e., a note's location) so anything to do with location (and therefore beat and meter) should be used cautiously. For instance, notice the difference in output if looking for the offset position of notes within a measure (which should correspond to their beat positions minus 1).

In [ ]:
offsets = []
for f in myfiles:
    for m in myfiles[0].parts[0].getElementsByClass('Measure'):
        for n in m.notes:
            offsets.append(n.offset)
    

In [ ]:
Counter(offsets)

Let's try with a piece from the built-in corpus instead:

In [ ]:
#note that each of these is a group of pieces that have been pre-assembled into a single piece
favBach = corpus.parse('bwv66.6')

In [ ]:
beatOnsets = []
timesigs = []
for i in favBach.flat.notes:
    beatOnsets.append(i.beat)

In [ ]:
Counter(beatOnsets)

In [ ]:
p2 = corpus.parse('chopin/mazurka06-2.krn')
p2.show()

In [ ]:
beatOnsets = []
timesigs = []
for i in p2.flat.notes:
    beatOnsets.append(i.beat)

In [ ]:
Counter(beatOnsets)

In [ ]:
#triplets on beat one: 3rd triplet in group
5/3

In [ ]:
#triplets on beat one: 2nd triplet in group
4/3

In [ ]:
#triplets on beat one: triplet 16th on second-to-last in group (note - I think this is an error)
11/6

### Exercise 4

In addition to `.beat` notes also have a `beatStr` attribute. Examine the distribution of `beatStrength` in the file:
"joplin/maple_leaf_rag.mxl" which is in the music21 corpus. What beat strength is the most common? Compare this output to the beat strengh profile of the "bwv66.6" file.

### Bonus:
Go back to Exercise #3 and, in conjunction with the "Intro_music21" notebook, add the analysis under the reduction in the lyrics attribute.